In [ ]:
!pip install transformers
!pip install underthesea

In [ ]:
# 1. Tiền xử lý ngôn ngữ
import re
from underthesea import word_tokenize

# Hàm chuẩn hoá câu
def standardize_data(row):
    # Xóa dấu kết thúc đặc biệt ở cuối câu
    row = re.sub(r"[\?]+$-", "", row)
    # Xóa tất cả dấu chấm phẩy, chấm thang, ... trong câu
    row = row.replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")
    row = row.strip()
    return row

def preprocess(sentence):
    # Chuẩn hoá câu
    sentence = standardize_data(sentence)
    # Phân tách từ
    segmented_sentence = word_tokenize(sentence, format="text")
    return segmented_sentence


In [ ]:
sentence = "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."
# Hiển thị câu đã phân tách
print(preprocess(sentence))

Ông Nguyễn_Khắc_Chúc đang làm_việc tại Đại_học Quốc_gia Hà_Nội . Bà_Lan , vợ ông Chúc , cũng làm_việc tại đây .


In [ ]:
# 2. Tải model
# import các thư viện cần thiết
import torch
import numpy as np
from transformers import RobertaForSequenceClassification, AutoTokenizer

# tải model và tokenizer

model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")

tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)


In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
# So sánh với model phoBert
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [ ]:
phobert

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [ ]:
# 3. Phân tích cảm xúc
def sentiment_analyst(model, tokenizer, sentence):
    # tokenize
    sentence = preprocess(sentence)
    print(sentence)
    input_ids = torch.tensor([tokenizer.encode(sentence)])
    with torch.no_grad():
        out = model(input_ids)
        probality = out.logits.softmax(dim=-1).tolist()
        predict = max(probality[0])
        predict_index = probality[0].index(predict)
        return predict, predict_index

In [ ]:
label = {
    0: 'Tiêu cực',
    1: 'Tích cực',
    2: 'Trung lập'
}

In [ ]:
# Câu cần phân tích cảm xúc
sentence = 'Đây là mô_hình rất hay phù_hợp với điều_kiện và như cầu của nhiều người'

# Tiến hành phân tích
predict, predict_index = sentiment_analyst(model, tokenizer, sentence)

# in ra kết quả phân tích
print(f'Phân tích cảm xúc của câu "{sentence}":\n{label[predict_index]} với tỉ lệ {round(predict*100, 2)}%. ')

Đây là mô_hình rất hay phù_hợp với điều_kiện và như cầu của nhiều người
Phân tích cảm xúc của câu "Đây là mô_hình rất hay phù_hợp với điều_kiện và như cầu của nhiều người":
Tích cực với tỉ lệ 98.67%. 


In [ ]:
# Câu cần phân tích cảm xúc
sentence = 'Tôi không thích chiếc điện thoại này'

# Tiến hành phân tích
predict, predict_index = sentiment_analyst(model, tokenizer, sentence)

# in ra kết quả phân tích
print(f'Phân tích cảm xúc của câu "{sentence}":\n{label[predict_index]} với tỉ lệ {round(predict*100, 2)}%. ')

Tôi không thích chiếc điện_thoại này
Phân tích cảm xúc của câu "Tôi không thích chiếc điện thoại này":
Tiêu cực với tỉ lệ 98.8%. 


In [ ]:
# Câu cần phân tích cảm xúc
sentence = 'ngôi nhà đang được xây'

# Tiến hành phân tích
predict, predict_index = sentiment_analyst(model, tokenizer, sentence)

# in ra kết quả phân tích
print(f'Phân tích cảm xúc của câu "{sentence}":\n{label[predict_index]} với tỉ lệ {round(predict*100, 2)}%. ')

ngôi nhà đang được xây
Phân tích cảm xúc của câu "ngôi nhà đang được xây":
Trung lập với tỉ lệ 53.22%. 


In [ ]:
# Câu cần phân tích cảm xúc
sentence = 'Cậu ấy đá banh giỏi quá'

# Tiến hành phân tích
predict, predict_index = sentiment_analyst(model, tokenizer, sentence)

# in ra kết quả phân tích
print(f'Phân tích cảm xúc của câu "{sentence}":\n{label[predict_index]} với tỉ lệ {round(predict*100, 2)}%. ')

Cậu ấy đá banh giỏi quá
Phân tích cảm xúc của câu "Cậu ấy đá banh giỏi quá":
Tích cực với tỉ lệ 84.49%. 


In [ ]:
# Câu cần phân tích cảm xúc
sentence = 'Cậu ấy đang buồn kìa'

# Tiến hành phân tích
predict, predict_index = sentiment_analyst(model, tokenizer, sentence)

# in ra kết quả phân tích
print(f'Phân tích cảm xúc của câu "{sentence}":\n{label[predict_index]} với tỉ lệ {round(predict*100, 2)}%. ')

Cậu ấy đang buồn kìa
Phân tích cảm xúc của câu "Cậu ấy đang buồn kìa":
Tiêu cực với tỉ lệ 95.8%. 


In [ ]:

# Câu cần phân tích cảm xúc
sentence = 'Bình thường. Không có gì đặc biệt.'

# Tiến hành phân tích
predict, predict_index = sentiment_analyst(model, tokenizer, sentence)

# in ra kết quả phân tích
print(f'Phân tích cảm xúc của câu "{sentence}":\n{label[predict_index]} với tỉ lệ {round(predict*100, 2)}%. ')

Bình_thường . Không có gì đặc_biệt .
Phân tích cảm xúc của câu "Bình thường. Không có gì đặc biệt.":
Trung lập với tỉ lệ 90.29%. 
